# TP Sytèmes de recommandation

## 1 Présentation du modèle

### Question 1.1

#### Que fait l'option `minidata` de la fonction `load_movielens` de `movielensutils.py` ?


In [ ]:
    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

L'option `minidata` réduit les données de travail aux notes des 100 premiers utilisateurs pour les 200 premiers films.

### Question 1.2

#### Combien y a-t-il d'utilisateurs, de films référencés dans la base de données ? Quel est le nombre total de notes ?

Il y a :
* 943 utilisateurs : nombre de lignes de R
* 1682 films : nombre de colonnes de R
* 100 000 notes : donné par `print(np.sum(m_mask))`

### Question 1.3

#### La fonction objectif est-elle convexe ?

Soit $f$ la fonction objectif :
$$
f: (P, Q) \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - QP)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q\|_F^2
$$
On remaque que $f$ est une forme quadratique.

Fixons $\rho = 0$ et $|U| = |C| = |I| = 1$. $f$ s'écrit alors : $f(p, q) = \frac{1}{2}(r - qp)^2$.

Soit $H(f)$ la matrice hessienne de f :
$$
H(f) =
\begin{bmatrix}
q^2 & 2qp -r \\
2qp -r & p^2
\end{bmatrix}
$$
Donc
$$
det(H(f)) = p^2q^2 - (2pq - r)^2 = (r - pq)(3pq - r)
$$

Pour $p = q = 0$ et $r = 1$, $det(H(f)) = -1$, la hessienne n'est donc pas définie positive, $f$ n'est donc pas convexe.

#### Quel est son gradient ?

$$
\nabla{f}(P,Q) =
\begin{bmatrix}
\nabla{f_Q} \\
\nabla{f_P}
\end{bmatrix}
= \begin{bmatrix}
-Q^T(\mathbb{1}_K \circ (R - QP)) + \rho P \\
(\mathbb{1}_K \circ (R-QP))(-P^T) + \rho Q
\end{bmatrix}
$$

#### Est-il lipschitzien ? Donner la constance de Lipschitz le cas échéant.

Si le gradient était Lipschitzien la hessienne serait bornée, or ici la hessienne est un polynôme d'ordre 2, $f$ n'est donc pas de gradient Lipschitzien.

## 2 Trouver $P$ quand $Q_0$ est fixé
$$
g: P \mapsto \frac{1}{2}\|\mathbb{1}_K \circ (R - Q^0P)\|_F^2 + \frac{\rho}{2}\|P\|_F^2 + \frac{\rho}{2}\|Q^0\|_F^2
$$

### Question 2.1

#### La fonction objectif $g$ est-elle convexe ?

# TODO

$P \mapsto \frac{\rho}{2}\|P\|_F^2$ est convexe.

#### Quel est son gradient ?

$$
\nabla{g}(P) = -(Q^0)^T(\mathbb{1}_K \circ (R-Q^0P)) + \rho P
$$

### Question 2.2

In [6]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = -Q0.transpose().dot(tmp) + rho*P

    return val, grad_P

### Question 2.3

In [ ]:
def gradient(g, P0, gamma, epsilon):
    """
    Minimise g par la méthode du gradient a pas constant.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    gamma : pas constant
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk inferieur ou egal a epsilon
    
    Sorties :
    Pk : minimiseur
    """
    
    grad_g = g(P0)[1]
    while np.sum(grad_g**2) > epsilon:
        Pk = Pk - gamma*grad_G
        grad_G = g(Pk)[1]
    
    return Pk

In [ ]:
def g(P):
    return objective(P, Q0, R, mask, rho)

L = rho + np.sum(Q0.transpose().dot(Q0)**2)
gradient(g, P0, 1/L, epsilon)

### Question 2.4

$\epsilon = 1$

## 3 Raffinements algorithmiques pour le problème à $Q_0$ fixé

### Question 3.1

In [ ]:
def gradient_linear(g, P0, epsilon):
    """
    Minimise g par la méthode du gradient avec recherche lineaire.
    Prend en entree
    g : fonction a minimiser, Pk -> g(Pk), grad_g(Pk)
    P0 : point de depart
    epsilon : critere d'arret, norme de Frobenius du gradient de g en Pk inferieur ou egal a epsilon

    Sorties :
    Pk : minimiseur
    """

    a, b, beta = 1/2, 1/2, 10**(-4) # Armijo's linear search parameters

    g_Pk, grad_g = g(P0)[1]
    while np.sum(grad_g**2) > epsilon:
        # Armijo's linear search
        l = 0
        while g(Pk-b*(a^l)*grad_g) > g(Pk) - beta*b*(a^l)*np.sum(grad_g**2):
            l++

        Pk = Pk - b*(a^l)*grad_g
        g_Pk, grad_g = g(Pk)[1]
    
    return Pk
    